<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Introducción


# 2.Armado del ambiente


##2.1 Armado del ambiente en CPU

No se requeriere ejecuciones previas para la ejecucion del programa


##2.2 Armado del ambiente en GPU

In [ ]:
!pip install pycuda

Se requiere la insatalacion del modulo de Cuda de Python

# 3.Desarrollo


In [ ]:
#--------------------------------------------------------------
#@title 3.1 Parametros de ejecución { vertical-output: true }

cantidad_N =  1000000#@param {type: "number"}

import numpy

import time

tiempo_total = time.time() * 1000

vector = numpy.random.randint(1000, size=cantidad_N+1)

def shellSort(arr): 
    n = len(arr) 
    gap = n//2
    
    while gap > 0: 
  
        for i in range(gap, n): 
            temp = arr[i] 
            j = i 
            while  j >= gap and arr[j-gap] >temp: 
                arr[j] = arr[j-gap] 
                j -= gap 
  
            arr[j] = temp 
        gap //= 2
  
 
print ("Original:") 
print(vector) 

tiempo_bucle = time.time() * 1000

shellSort(vector) 

tiempo_bucle = (time.time() * 1000) - tiempo_bucle   

print ("Ordenado:") 
print(vector)

tiempo_total = (time.time() * 1000) - tiempo_total

print("Tiempo Total: ", tiempo_total , "[ms]" )
print("Tiempo bucle: ", tiempo_bucle , "[ms]" )  

  

Original:
[969 364 643 ... 290  31 357]
Ordenado:
[  0   0   0 ... 999 999 999]
Tiempo Total:  25076.254638671875 [ms]
Tiempo bucle:  25063.718505859375 [ms]


#4.Tabla de pasos


# 5.Conclusiones

# 6.Bibliografía
